In [2]:
%matplotlib widget

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd

In [3]:
#parameters
scan_id=3
sample_rate=10
no_of_lines =50
line_tolarance=1.5
model_name="svm"

In [4]:
def process_data(file, sample_rate = sample_rate):
    coord = []
    for point in file.readlines():
        coord.append(list(map(float,point.split())))
        
    point_array_sampled = []
    for i in range(0,len(coord),sample_rate):
        point_array_sampled.append(coord[i])
        
    point_array_sampled = np.array(point_array_sampled)
        
    return np.array( point_array_sampled)

In [5]:
def plot_graph(point_data,outline_points=None, lines=None):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create cubic bounding box to simulate equal aspect ratio
    X = point_data[:,0]
    Y = point_data[:,1]
    Z = point_data[:,2]

    ax.scatter(point_data[:,0], point_data[:,1], point_data[:,2], s=1)

    #bounding box to show foot in scale
    max_range = np.array([X.max()-X.min(), Y.max()-Y.min(), Z.max()-Z.min()]).max()
    Xb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][0].flatten() + 0.5*(X.max()+X.min())
    Yb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][1].flatten() + 0.5*(Y.max()+Y.min())
    Zb = 0.5*max_range*np.mgrid[-1:2:2,-1:2:2,-1:2:2][2].flatten() + 0.5*(Z.max()+Z.min())

    for xb, yb, zb in zip(Xb, Yb, Zb):
        ax.plot([xb], [yb], [zb], 'w')
        
    if (outline_points is not None):
        flatten = outline_points.reshape(no_of_lines*2,3)
        ax.scatter(flatten[:,0], flatten[:,1], flatten[:,2], s=500/no_of_lines, color='red')

    plt.grid()

In [6]:
def project_points(point_array, axis=1, height_limit=100000): #axis values x=0/y=1/z=2 implement limit
    if(axis==0):
        point_array[:,0]=0
    if(axis==1):
        point_array[:,1]=0
    if(axis==2):
        point_array[:,2]=0
        
    return point_array

In [7]:
def get_outline(point_data, no_of_lines = no_of_lines, line_tolarance = line_tolarance):
    
    z_min = min(point_data[:,2])
    z_max =max(point_data[:,2])
    
    step = (z_max-z_min)/(no_of_lines+1)
    
    lines = np.arange(z_min, z_max, step)[1:]
    
    outline_points=[]
    
    for line in lines:
        line_points=[]
        candidates = point_data[np.where(np.logical_and(point_data[:,2]>line, point_data[:,2]<line+line_tolarance))]
        
        if(len(candidates)==0 or len(outline_points)==no_of_lines):
            break
            
        line_points.append(candidates[np.where(candidates[:,0]==min(candidates[:,0]))][0])
        line_points.append(candidates[np.where(candidates[:,0]==max(candidates[:,0]))][0])
        
        
        outline_points.append(line_points)
        
    return np.array(outline_points).reshape((no_of_lines,2,3)),lines

In [8]:
def read_dataset():
    all_set = pd.read_csv("Datasets/all_data.csv", sep=',',header=0)
    train_set=pd.read_csv("Datasets/train_set.csv", sep=',',header=0)
    test_set=pd.read_csv("Datasets/test_set.csv", sep=',',header=0)
    
    return all_set.drop(all_set.index[[0,1]]),train_set.drop(train_set.index[[0,1]]), test_set.drop(test_set.index[[0,1]])

In [9]:
def load_model(model=model_name):
    if(model=="svm"):
        return pickle.load(open("Models/SVM_Classifier.pkcls", 'rb'))
    else:
        return

In [10]:
points_1 = open("Datasets/Foot Models/points/left/3/sub473ll.pts")
point_array_1 = process_data(points_1)
projection_1 = project_points(point_array_1)
outline_1 =get_outline(projection_1)
outline_points_1= outline_1[0]
lines_1=outline_1[1]

In [11]:
points_2 = open("Datasets/Foot Models/points/left/3/sub1673ll.pts")
point_array_2 = process_data(points_2)
projection_2 = project_points(point_array_2)
outline_2 =get_outline(projection_2)
outline_points_2= outline_2[0]
lines_2=outline_2[1]

In [12]:
points_3 = open("Datasets/Foot Models/points/left/3/sub113ll.pts")
point_array_3 = process_data(points_3)
projection_3 = project_points(point_array_3)
outline_3 =get_outline(projection_3)
outline_points_3= outline_3[0]
lines_3=outline_3[1]

In [13]:
plot_graph(projection_1, outline_points=outline_points_1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plot_graph(projection_2,outline_points=outline_points_2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plot_graph(projection_3,outline_points=outline_points_3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
dataset= read_dataset()
all_data= dataset[0]
train_set = dataset[1]
test_set = dataset[2]

In [17]:
model=load_model() #implement classification model
test_features=test_set[["No.","Cluster","Foot Length","Foot Breadth 1","Short Heel Girth"]]

In [18]:
print(len(outline_points_1),len(outline_points_2),len(outline_points_3))

50 50 50


In [19]:
outline_points_3[0][0]

array([-39.051582,   0.      , 127.890007])

In [24]:
# genetic algorithm search for continuous function optimization
from numpy.random import randint
from numpy.random import rand

# objective function to minimize
def objective(x):
    func = np.linalg.norm((x[0]*np.array([0,1]) + x[1]*np.array([3,4]))-np.array([2,1])) + np.linalg.norm((x[0]*np.array([0,1]) + x[1]*np.array([3,4]))-np.array([2,1]))
    return func

# decode bitstring to numbers
def decode(bounds, n_bits, bitstring):
    decoded = list()
    largest = 2**n_bits
    for i in range(len(bounds)):
        # extract the substring
        start, end = i * n_bits, (i * n_bits)+n_bits
        substring = bitstring[start:end]
        # convert bitstring to a string of chars
        chars = ''.join([str(s) for s in substring])
        # convert string to integer
        integer = int(chars, 2)
        # scale integer to desired range
        value = bounds[i][0] + (integer/largest) * (bounds[i][1] - bounds[i][0])
        # store
        decoded.append(value)
    return decoded

# tournament selection
def selection(pop, scores, k=3):
    # first random selection
    selection_ix = randint(len(pop))
    for ix in randint(0, len(pop), k-1):
        # check if better (e.g. perform a tournament)
        if scores[ix] < scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    if rand() < r_cross:
        # select crossover point that is not on the end of the string
        pt = randint(1, len(p1)-2)
        # perform crossover
        c1 = p1[:pt] + p2[pt:]
        c2 = p2[:pt] + p1[pt:]
    return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
    for i in range(len(bitstring)):
        # check for a mutation
        if rand() < r_mut:
            # flip the bit
            bitstring[i] = 1 - bitstring[i]

# genetic algorithm
def genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut):
    # initial population of random bitstring
    pop = [randint(0, 2, n_bits*len(bounds)).tolist() for _ in range(n_pop)]
    # keep track of best solution
    best, best_eval = 0, objective(decode(bounds, n_bits, pop[0]))
    # enumerate generations
    for gen in range(n_iter):

        # decode population
        decoded = [decode(bounds, n_bits, p) for p in pop]
        # evaluate all candidates in the population
        scores = [objective(d) for d in decoded]
        # check for new best solution
        for i in range(n_pop):
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
                print(">%d, new best f(%s) = %f" % (gen,  decoded[i], scores[i]))
        # select parents
        selected = [selection(pop, scores) for _ in range(n_pop)]
        # create the next generation
        children = list()
        for i in range(0, n_pop, 2):
            # get selected parents in pairs
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            for c in crossover(p1, p2, r_cross):
                # mutation
                mutation(c, r_mut)
                # store for next generation
                children.append(c)
        # replace population
        pop = children
    return [best, best_eval]

# define range for input
bounds = [[-5,5], [-5,5]]
# define the total iterations
n_iter = 1000
# bits per variable
n_bits = 16
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / (float(n_bits) * len(bounds))
# perform the genetic algorithm search
best, score = genetic_algorithm(objective, bounds, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
decoded = decode(bounds, n_bits, best)
print('f(%s) = %f' % (decoded, score))


#set bound to (-1 to 1) and normalize the foots to max length foot

>0, new best f([4.43634033203125, 1.26129150390625]) = 17.334146
>0, new best f([2.320556640625, -0.7501220703125]) = 9.140631
>0, new best f([2.701263427734375, 0.0628662109375]) = 5.327034
>0, new best f([-1.920166015625, 0.675048828125]) = 0.442807
>1, new best f([-1.920166015625, 0.67535400390625]) = 0.440594
>3, new best f([-1.920166015625, 0.71441650390625]) = 0.312581
>4, new best f([-1.73126220703125, 0.69305419921875]) = 0.178258
>5, new best f([-1.70623779296875, 0.676116943359375]) = 0.056812
>6, new best f([-1.70623779296875, 0.67596435546875]) = 0.055989
>9, new best f([-1.654052734375, 0.6658935546875]) = 0.019600
>12, new best f([-1.65435791015625, 0.6658935546875]) = 0.019007
>13, new best f([-1.65924072265625, 0.6658935546875]) = 0.009830
>21, new best f([-1.65985107421875, 0.6658935546875]) = 0.008773
>22, new best f([-1.67327880859375, 0.66802978515625]) = 0.008501
>24, new best f([-1.66351318359375, 0.665435791015625]) = 0.008190
>24, new best f([-1.67327880859375, 